In [1]:
md_reference_data = None
upstream = None
product = None

# R23 - MD alignment and RMSD extraction 

## Contents

1. Alignment
2. RMSD
3. RMSD against average structure
4. RMSD over subunit
5. RMSF

## Alignment and preparation

In [2]:
from mdtools.mapping import map_alignment_to_structure, align_structure_sequences
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align
from pyfamsa import Aligner, Sequence
import json
import pandas as pd
import prody as pdy
#matplotlib.rcParams['fontname'] = "Arial"

/home/kaustubh/miniconda3/envs/md-analysis/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [3]:
def load_trajectory(code, data, path):
    
    """
    loads the following files
    - an MDAnalysis trajectory with the dried (water-removed) dcd
    - an MDAnalysis Universe with the starting PDB (jic)
    - an prody with the starting PDB (pretty useful when MDAnalysis fails)
    """
    return dict(
        code=code,
        trajectory_dry = mda.Universe(
            path + data['dry_pdb'], 
            path + data['dry_dcd'], 
            frames='all', in_memory=True
        ),
        reference_pdy=pdy.parsePDB(path + data['dry_pdb']),
        reference=mda.Universe(
            path + data['dry_pdb']
        ),
        time=data['time']
    )


Loading all the simulations.

In [ ]:
path = '../../../simulations/'
exp01_md = []
for key, items in md_reference_data.items():
    print(f"-- code {key}", end='')
    exp01_md.append(load_trajectory(
        code=key, data=items, path=path
    ))
    print(f" loaded")
exp01_md = pd.DataFrame.from_records(exp01_md).set_index('code')

## RMSD calculations

We compute the RMSD of:
- All the alpha carbons.
- Large subunit alpha carbons
- Small subunit alpha carbons
- Upper and lower large subunit alpha carbons
- Upper and lower small subunit alpha carbons

In [ ]:
exp01_rmsd = []

upper_ssu = 'name CA and (segid B or segid F or segid J or segid N)'
lower_ssu = 'name CA and (segid D or segid H or segid L or segid P)'
ssu = 'name CA and (segid B or segid F or segid J or segid N or segid D or segid H or segid L or segid P)'
upper_lsu = 'name CA and (segid A or segid E or segid I or segid M)'
lower_lsu = 'name CA and (segid C or segid G or segid K or segid O)'
lsu = 'name CA and (segid A or segid E or segid I or segid M or segid C or segid G or segid K or segid O)'


for key, item in exp01_md.iterrows():
    print(f"-- code {key}", end='')
    R = rms.RMSD(item['trajectory_dry'], item['reference'], select='name CA', groupselections=[upper_ssu, lower_ssu, ssu, upper_lsu, lower_lsu, lsu], superposition=False)
    results = R.run()
    tmp = pd.DataFrame(results.rmsd, columns=['frame', 'time', 'CA', 'upperSSU', 'lowerSSU', 'ssu', 'upperLSU', 'lowerLSU', 'lsu'])
    tmp['code'] = key
    exp01_rmsd.append(tmp)
    print(f" aligned")
    
exp01_rmsd = pd.concat(exp01_rmsd)
exp01_rmsd.index = pd.MultiIndex.from_arrays([exp01_rmsd['code'], exp01_rmsd['frame']])
exp01_rmsd['frame'] = exp01_rmsd['frame'].astype(int)
exp01_rmsd['time'] = exp01_rmsd['time'].astype(float)
exp01_rmsd['time_ns'] = exp01_rmsd['time'] / 10

We only make measurements using the last 50ns of each simulation, counting the previous 25 to 50ns as burn-in time.

In [ ]:
exp01_rmsd['time_to_end'] = exp01_rmsd.groupby(level='code')['time_ns'].transform(
    lambda x: x.max() - x
)
exp01_rmsd.query('time_to_end < 50')['time_ns']

Dumping values to file.

In [ ]:
exp01_rmsd.to_csv(product['rmsd'], index=None)

### RMSD over subunits

We compute the average structure RMSD by subunits.

First, we need to generate all the average subunits.

In [ ]:
lsu_A = 'name CA and segid A'
lsu_E = 'name CA and segid E'
lsu_I = 'name CA and segid I'
lsu_M = 'name CA and segid M'
lsu_C = 'name CA and segid C'
lsu_G = 'name CA and segid G'
lsu_K = 'name CA and segid K'
lsu_O = 'name CA and segid O'

ssu_B = 'name CA and segid B'
ssu_F = 'name CA and segid F'
ssu_J = 'name CA and segid J'
ssu_N = 'name CA and segid N'
ssu_D = 'name CA and segid D'
ssu_H = 'name CA and segid H'
ssu_L = 'name CA and segid L'
ssu_P = 'name CA and segid P'

lsu = 'name CA and (segid A or segid E or segid I or segid M or segid C or segid G or segid K or segid O)'
ssu = 'name CA and (segid B or segid F or segid J or segid N or segid D or segid H or segid L or segid P)'

selection_strings = dict(
    overall = 'name CA',
    ssu=ssu, 
    lsu=lsu,
    lsu_A = lsu_A,
    lsu_E = lsu_E,
    lsu_I = lsu_I,
    lsu_M = lsu_M,
    lsu_C = lsu_C,
    lsu_G = lsu_G,
    lsu_K = lsu_K,
    lsu_O = lsu_O,
    ssu_B = ssu_B,
    ssu_F = ssu_F,
    ssu_J = ssu_J,
    ssu_N = ssu_N,
    ssu_D = ssu_D,
    ssu_H = ssu_H,
    ssu_L = ssu_L,
    ssu_P = ssu_P
)
average_structures = dict(
    overall=[],
    ssu=[],
    lsu=[],
    lsu_A=[],
    lsu_E=[],
    lsu_I=[],
    lsu_M=[],
    lsu_C=[],
    lsu_G=[],
    lsu_K=[],
    lsu_O=[],
    ssu_B=[],
    ssu_F=[],
    ssu_J=[],
    ssu_N=[],
    ssu_D=[],
    ssu_H=[],
    ssu_L=[],
    ssu_P=[]
)

for key, item in exp01_md.iterrows():
    print(f"-- code {key}", end='')
    for selection_key, selection_string in selection_strings.items():
        try:
            average = align.AverageStructure(
                item['trajectory_dry'], item['trajectory_dry'], 
                select=selection_string, ref_frame=0
            ).run()
        except ValueError:
            average_structures[selection_key].append(None)
            continue

        ref = average.results.universe
        average_structures[selection_key].append(ref)
    print(" aligned")

exp01_md['overall_avg'] = average_structures['overall']
exp01_md['ssu_avg'] = average_structures['ssu']
exp01_md['lsu_avg'] = average_structures['lsu']

exp01_md['lsu_A'] = average_structures['lsu_A']
exp01_md['lsu_E'] = average_structures['lsu_E']
exp01_md['lsu_I'] = average_structures['lsu_I']
exp01_md['lsu_M'] = average_structures['lsu_M']
exp01_md['lsu_C'] = average_structures['lsu_C']
exp01_md['lsu_G'] = average_structures['lsu_G']
exp01_md['lsu_K'] = average_structures['lsu_K']
exp01_md['lsu_O'] = average_structures['lsu_O']

exp01_md['ssu_B'] = average_structures['ssu_B']
exp01_md['ssu_F'] = average_structures['ssu_F']
exp01_md['ssu_J'] = average_structures['ssu_J']
exp01_md['ssu_N'] = average_structures['ssu_N']
exp01_md['ssu_D'] = average_structures['ssu_D']
exp01_md['ssu_H'] = average_structures['ssu_H']
exp01_md['ssu_L'] = average_structures['ssu_L']
exp01_md['ssu_P'] = average_structures['ssu_P']

Computing the RMSD over the different subunit structures.

In [ ]:
exp01_rmsd_avg = []
overall = 'name CA'

for key, item in exp01_md.iterrows():
    print(f"-- code {key}", end='')
    R = rms.RMSD(item['trajectory_dry'], item['overall_avg'], select='name CA', groupselections=[ssu, lsu, lsu_A, lsu_E, lsu_I, lsu_M, lsu_C, lsu_G, lsu_K, lsu_O, ssu_B, ssu_F, ssu_J, ssu_N, ssu_D, ssu_H, ssu_L, ssu_P], superposition=True)
    results = R.run()
    tmp = pd.DataFrame(results.rmsd, columns=['frame', 'time', 'CA', 'ssu', 'lsu', 'lsu_A', 'lsu_E', 'lsu_I', 'lsu_M', 'lsu_C', 'lsu_G', 'lsu_K', 'lsu_O', 'ssu_B', 'ssu_F', 'ssu_J', 'ssu_N', 'ssu_D', 'ssu_H', 'ssu_L', 'ssu_P'])
    tmp['code'] = key
    
    exp01_rmsd_avg.append(tmp)
    print(" aligned")
    
exp01_rmsd_avg = pd.concat(exp01_rmsd_avg)
exp01_rmsd_avg.index = pd.MultiIndex.from_arrays([exp01_rmsd_avg['code'], exp01_rmsd_avg['frame']])
exp01_rmsd_avg['frame'] = exp01_rmsd_avg['frame'].astype(int)
exp01_rmsd_avg['time'] = exp01_rmsd_avg['time'].astype(float)
exp01_rmsd_avg['time_ns'] = exp01_rmsd_avg['time'] / 10
exp01_rmsd_avg['time_to_end'] = exp01_rmsd_avg.groupby(level='code')['time_ns'].transform(
    lambda x: x.max() - x
)
exp01_rmsd_avg.query('time_to_end < 50')['time_ns']

In [ ]:
exp01_rmsd_avg.to_csv(
    product['avg_rmsd'], index=None
)